In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


batch_size = 128
num_classes = 10
epochs = 12


img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest')


model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=2)
score = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


model.save('model.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/12
468/468 - 171s - loss: 1.1466 - accuracy: 0.6128 - val_loss: 0.1836 - val_accuracy: 0.9498 - 171s/epoch - 365ms/step
Epoch 2/12
468/468 - 196s - loss: 0.6064 - accuracy: 0.8079 - val_loss: 0.1223 - val_accuracy: 0.9630 - 196s/epoch - 419ms/step
Epoch 3/12
468/468 - 168s - loss: 0.4693 - accuracy: 0.8546 - val_loss: 0.0971 - val_accuracy: 0.9693 - 168s/epoch - 359ms/step
Epoch 4/12
468/468 - 163s - loss: 0.4061 - accuracy: 0.8751 - val_loss: 0.0963 - val_accuracy: 0.9689 - 163s/epoch - 349ms/step
Epoch 5/12
468/468 - 161s - loss: 0.3574 - accuracy: 0.8914 - val_loss: 0.0908 - val_accuracy: 0.9724 - 161s/epoch - 344ms/step
Epoch 6/12
468/468 - 161s - loss: 0.3270 - accuracy: 0.9009 - val_loss: 0.0909 - val_accuracy: 0.9730 - 161s/epoch - 344ms/step
Epoch 7/12
468/468 - 161s - loss: 0.3047 - accuracy: 0.9096 - val_loss: 0.0808 - val_accuracy: 0.9744 - 161s/epoch - 343ms/step
Epoch 8/12
468/468 - 160s - loss: 0.2891 - accuracy: 0.9137 - val_loss: 0.0640 - val_accuracy: 0.9804 - 

In [ ]:
import numpy as np
import cv2 as cv
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow
from google.colab import drive


img_color = cv.imread('/content/drive/MyDrive/crop/tt.jpg', cv.IMREAD_COLOR)
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)


ret,img_binary = cv.threshold(img_gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)

kernel = cv.getStructuringElement( cv.MORPH_RECT, ( 5, 5 ) )
img_binary = cv.morphologyEx(img_binary, cv. MORPH_CLOSE, kernel)

cv2_imshow(img_binary)

contours, hierarchy = cv.findContours(img_binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

for contour in contours:

    x, y, w, h = cv.boundingRect(contour)



    length = max(w, h) + 60
    img_digit = np.zeros((length, length, 1),np.uint8)

    new_x,new_y = x-(length - w)//2, y-(length - h)//2


    img_digit = img_binary[new_y:new_y+length, new_x:new_x+length]

    kernel = np.ones((5, 5), np.uint8)
    img_digit = cv.morphologyEx(img_digit, cv.MORPH_DILATE, kernel)

    cv2_imshow(img_digit)

    model = load_model('model.h5')

    img_digit = cv.resize(img_digit, (28, 28), interpolation=cv.INTER_AREA)

    img_digit = img_digit / 255.0

    img_input = img_digit.reshape(1, 28, 28, 1)
    predictions = model.predict(img_input)


    number = np.argmax(predictions)
    print(number)

    cv.rectangle(img_color, (x, y), (x+w, y+h), (255, 255, 0), 1)


    location = (x + int(w *0.5), y - 10)
    font = cv.FONT_HERSHEY_COMPLEX
    fontScale = 1.2
    cv.putText(img_color, str(number), location, font, fontScale, (0,255,0), 2)


cv2_imshow(img_color)